<p align="center"><img src="logo/spark.png" alt="Hadoop Logo" width="250"/></p>
# **Lab 2 - Part 1 - Spark**
#### The following steps demonstrate how to create a simple Spark application in Scala. In this notebook you will see how to make a base RDD and appy functions to it.



### ** Part 1: Warm Up **

Create a collection of integers in the range of 1 ... 10000.

In [ ]:
// TODO: Replace <FILL IN> with appropriate code
val data = <FILL IN>
print(data)

Use that collection to create a base RDD.

In [ ]:
// TODO: Replace <FILL IN> with appropriate code
val distData = <FILL IN>
print(distData)

Namely a `filter()` transformation to keep the values less than 10, then a `collect()` action to collect the results.

In [ ]:
val x = distData.filter(<FILL IN>)
x.<FILL IN>

### ** Part 2: Create an RDD From a File **

The following steps demonstrate how to create an RDD from a file and apply transofrmations on it in Scala. Creat an RDD, named `pagecounts`, from the input files at `data/pagecounts`. The files entries will look something like this:
```
20090507-040000 zh favicon.ico 67 62955
```

In [ ]:
// TODO: Replace <FILL IN> with appropriate code
val pagecounts = <FILL IN>
print(pagecounts)

Use the `take()` operation of an RDD to get the first 10 records.

In [ ]:
// TODO: Replace <FILL IN> with appropriate code
pagecounts.<FILL IN>

An alternative way to print the fields is to travers the array and print each record on its own line.

In [ ]:
// TODO: Replace <FILL IN> with appropriate code
for (<FILL IN>) {
   println(x)
}

Use the `count()` function to see how many records in total are in this data set. The `pagecounts` folder consists of two files, each with around 700K lines, so in total we have around 1400K lines.

In [ ]:
// TODO: Replace <FILL IN> with appropriate code
pagecounts.<FILL IN>

The second field of each record in the data set is the "project code" and contains information about the language of the pages. For example, the project code "en" indicates an English page. Let's derive an RDD, named `enPages`, containing only English pages from pagecounts. This can be done by applying a `filter()` function to pagecounts. For each record, we can split it by the field delimiter (i.e., a space) and get the second field, and then compare it with the string "en". To avoid reading from disks each time we perform any operations on the RDD, we can use `cache()` to cache the RDD into memory. 

In [ ]:
// TODO: Replace <FILL IN> with appropriate code
val enPages = pagecounts.filter(<FILL IN>).cache()

The above command defines the RDD, but because of lazy evaluation, no computation is done yet. Next time any action is invoked on `enPages`, Spark will cache the data set in memory across the workers in your cluster. So, let's count the number of records, which are there for English pages. The first time this command is run, it will take 2-3 minutes while Spark scans through the entire data set on disk. But since `enPages` was marked as "cached" in the previous step, if you run count on the same RDD again, it should return an order of magnitude faster.

In [ ]:
// TODO: Replace <FILL IN> with appropriate code
enPages.<FILL IN>

Let's generate a histogram of total page views on Wikipedia English pages for the date range represented in our dataset (May 5 to May 7, 2009). The high level idea of what we'll be doing is as follows. First, we generate a key value pair for each line; the key is the date (the first eight characters of the first field), and the value is the number of pageviews for that date (the fourth field).

In [ ]:
// TODO: Replace <FILL IN> with appropriate code
val enTuples = enPages.map(<FILL IN>)
val enKeyValuePairs = enTuples.map(<FILL IN>)

Next, we shuffle the data and group all values of the same key together. Finally we sum up the values for each key. There is a convenient method called `reduceByKey` in Spark for exactly this pattern. Note that the second argument to `reduceByKey` determines the number of reducers to use. By default, Spark assumes that the reduce function is commutative and associative and applies combiners on the mapper side. Since we know there is a very limited number of keys in this case (because there are only 3 unique dates in our data set), let’s use only one reducer.

In [ ]:
// TODO: Replace <FILL IN> with appropriate code
enKeyValuePairs.reduceByKey(<FILL IN>, 1).collect()

The `collect()` method at the end converts the result from an RDD to an array. We can combine the previous three commands into one:

In [ ]:
// TODO: Replace <FILL IN> with appropriate code
enPages.map(<FILL IN>).map(<FILL IN>).reduceByKey(<FILL IN>, 1).collect()

Suppose we want to find pages that were viewed more than 200,000 times during the three days covered by our dataset. Conceptually, this task is similar to the previous query. But, given the large number of pages (23 million distinct page names), the new task is very expensive. We are doing an expensive group-by with a lot of network shuffling of data. To recap, first we split each line of data into its respective fields. Next, we extract the fields for page name and number of page views. We reduce by key again, this time with 40 reducers. Then we filter out pages with less than 200,000 total views over our time window represented by our dataset.

In [ ]:
// TODO: Replace <FILL IN> with appropriate code
enPages.map(x => x.split(" ")).map(<FILL IN>).filter(<FILL IN>)
.reduceByKey(<FILL IN>, <FILL IN>).map(x => (x._2, x._1)).collect()